In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
n_element = 10

element_value = pd.Series(
    np.arange(n_element),
    name="Value",
    index=pd.Index(("Element{}".format(i) for i in range(n_element)), name="Element"),
)

set_elements = element_value.index[-2:]

for method in (
    ("rank", "cdf", "s0", "supreme"),
    ("rank", "cdf", "s1", "supreme"),
    ("rank", "cdf", "s2", "supreme"),
    ("value", "pdf", "s0", "supreme"),
    ("value", "pdf", "s1", "supreme"),
    ("value", "pdf", "s2", "supreme"),
    ("value", "cdf", "s0", "supreme"),
    ("value", "cdf", "s1", "supreme"),
    ("value", "cdf", "s2", "supreme"),
    ("value_", "pdf", "s0", "supreme"),
    ("value_", "pdf", "s1", "supreme"),
    ("value_", "pdf", "s2", "supreme"),
    ("value_", "cdf", "s0", "supreme"),
    ("value_", "cdf", "s1", "supreme"),
    ("value_", "cdf", "s2", "supreme"),
):

    kraft.score_set(element_value, set_elements, method=method, plot_=False, plot=False)

In [ ]:
for n_element in (100,):

    element_value = np.random.normal(size=n_element // 2)

    element_value = pd.Series(
        np.sort(np.concatenate((-element_value, element_value))),
        name="Value",
        index=pd.Index(
            ("Element{}".format(i) for i in range(n_element)), name="Element"
        ),
    )

    for n_skip in (0.1,):

        if n_skip < 1:

            n_skip = int(n_skip * n_element)

        for n_top in (1,):

            if n_top < 1:

                n_top = int(n_top * n_element)

            for rate_top_hit in (1,):

                if rate_top_hit < 1:

                    rate_top_hit = int(rate_top_hit * n_top)

                for side in ("+",):

                    set_elements = []

                    if "-" in side:

                        set_elements += element_value.index[
                            n_skip : n_skip + n_top : rate_top_hit
                        ].tolist()

                    if "+" in side:

                        set_elements += element_value.index[
                            -(n_skip + 1) : -(n_skip + 1 + n_top) : -rate_top_hit
                        ].tolist()

                    for method in ("rank cdf s0 supreme",):

                        kraft.score_set(
                            element_value, set_elements, method=method,
                        )

In [ ]:
element_value = pd.read_csv("gene_value.tsv", sep="\t", index_col=0, squeeze=True)

set_elements = kraft.read_gmt("c2.gmt")["COLLER_MYC_TARGETS_UP"]

In [ ]:
kraft.score_set(element_value, set_elements, method="rank cdf s0 supreme", plot_=True)

In [ ]:
n = 100

vector = np.random.normal(size=n)

# vector = np.concatenate((vector, -vector))

vector = np.sort(vector)

# vector -= vector[0]

n_skip = 10

n_hit = 10

h_is = np.asarray([0] * n_skip + [1] * n_hit + [0] * (n - n_skip - n_hit))

# h_is = np.concatenate((h_is, h_is[::-1]))

m_is = 1 - h_is

h_i = np.where(h_is)[0]

m_i = np.where(m_is)[0]

kraft.plot_plotly(
    {
        "data": [
            {"x": h_i, "y": vector[h_i], "mode": "markers"},
            {"x": m_i, "y": vector[m_i], "mode": "markers"},
        ]
    }
)

h_v = vector * h_is

m_v = vector * m_is

kraft.plot_plotly({"data": [{"y": h_v, "mode": "lines"}, {"y": m_v, "mode": "lines"},]})


def get_c(vector):

    vector = np.absolute(vector)

    l_c = kraft.normalize(vector.cumsum(), "0-1")

    r_c = kraft.normalize(vector[::-1].cumsum()[::-1], "0-1")

    small_number = 1e-8

    l_c += small_number

    r_c += small_number

    return l_c, r_c


h_l_c, h_r_c = get_c(h_v)

m_l_c, m_r_c = get_c(m_v)

a_l_c, a_r_c = get_c(vector)

kraft.plot_plotly(
    {
        "data": [
            {"y": h_l_c, "mode": "lines"},
            {"y": m_l_c, "mode": "lines"},
            {"y": a_l_c, "mode": "lines"},
        ]
    }
)

kraft.plot_plotly(
    {
        "data": [
            {"y": h_r_c, "mode": "lines"},
            {"y": m_r_c, "mode": "lines"},
            {"y": a_r_c, "mode": "lines"},
        ]
    }
)

s_l_0 = h_l_c - m_l_c

s_l_1 = h_l_c * np.log(h_l_c / a_l_c) - m_l_c * np.log(m_l_c / a_l_c)

kraft.plot_plotly(
    {"data": [{"y": s_l_0, "mode": "lines"}, {"y": s_l_1, "mode": "lines"},]}
)

s_r_0 = h_r_c - m_r_c

s_r_1 = h_r_c * np.log(h_r_c / a_r_c) - m_r_c * np.log(m_r_c / a_r_c)

kraft.plot_plotly(
    {"data": [{"y": s_r_0, "mode": "lines"}, {"y": s_r_1, "mode": "lines"},]}
)

kraft.plot_plotly(
    {
        "data": [
            {"y": s_r_0 - s_l_0, "mode": "lines"},
            {"y": s_r_1 - s_l_1, "mode": "lines"},
        ]
    }
)